In [1]:
import cv2
import mediapipe as mp
import os
import numpy as np

In [3]:
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False,max_num_hands=2, min_detection_confidence=0.5)

I0000 00:00:1715549391.489834 4921729 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M2 Pro


W0000 00:00:1715549391.496207 4921937 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1715549391.502345 4921939 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


In [33]:
def process_video(video_path):
    cap = cv2.VideoCapture(video_path)
    max_hands = 2  # Maximum number of hands you want to detect per frame
    landmark_count = 21  # Number of landmarks per hand in MediaPipe
    video_results = []

    while cap.isOpened():
        success, image = cap.read()
        if not success:
            break

        results = hands.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        
        # Initialize frame data with placeholders for max_hands
        frame_data = np.full((max_hands, landmark_count, 3), -1.0, dtype=float)
        
        if results.multi_hand_landmarks:
            for i, hand_landmarks in enumerate(results.multi_hand_landmarks[:max_hands]):
                for j, lm in enumerate(hand_landmarks.landmark):
                    frame_data[i, j] = [lm.x, lm.y, lm.z]
        
        video_results.append(frame_data)

    cap.release()
    return np.array(video_results)

In [31]:
def save_results(results, output_path):
    np.save(output_path, np.array(results))

In [35]:
for video_name in os.listdir(video_folder):
    video_path = os.path.join(video_folder, video_name)
    video_results = process_video(video_path)
    npy_path = os.path.join(output_folder, f"{os.path.splitext(video_name)[0]}.npy")
    np.save(npy_path, video_results)

/opt/anaconda3/envs/myenv/lib/python3.11/site-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


Processed 52291.mp4 and saved results to /Users/ethanfarah/stanford-workspace/live-fingering/data/landmarks/52291.npy
Processed 34114.mp4 and saved results to /Users/ethanfarah/stanford-workspace/live-fingering/data/landmarks/34114.npy
Processed 60545.mp4 and saved results to /Users/ethanfarah/stanford-workspace/live-fingering/data/landmarks/60545.npy
Processed 17915.mp4 and saved results to /Users/ethanfarah/stanford-workspace/live-fingering/data/landmarks/17915.npy
Processed 51940.mp4 and saved results to /Users/ethanfarah/stanford-workspace/live-fingering/data/landmarks/51940.npy
Processed 39430.mp4 and saved results to /Users/ethanfarah/stanford-workspace/live-fingering/data/landmarks/39430.npy
Processed 69207.mp4 and saved results to /Users/ethanfarah/stanford-workspace/live-fingering/data/landmarks/69207.npy
Processed 51798.mp4 and saved results to /Users/ethanfarah/stanford-workspace/live-fingering/data/landmarks/51798.npy
Processed 38048.mp4 and saved results to /Users/ethanfar

In [38]:
def process_and_save(video_name):
    video_path = os.path.join(video_folder, video_name)
    video_results = process_video(video_path)
    npy_path = os.path.join(output_folder, f"{os.path.splitext(video_name)[0]}.npy")
    np.save(npy_path, video_results)
    return f"Processed and saved {video_name}"

Processed /: Shape (0,)


OpenCV: Couldn't read video stream from file "/"


IndexError: index 0 is out of bounds for axis 0 with size 0

In [ ]:
from multiprocessing import Pool

# Get the current working directory (where the script is run from)
base_dir = os.getcwd()

# Construct paths relative to the base directory
video_folder = os.path.join(base_dir, 'data', 'videos')
output_folder = os.path.join(base_dir, 'data', 'landmarks')

video_files = os.listdir(video_folder)

# Use all available cores, or specify the number of processes
with Pool(processes=os.cpu_count()) as pool:
    results = pool.map(process_and_save, video_files)
    for result in results:
        print(result)